In [ ]:
{
 "cells": [
  {
   "cell_type": "markdown",
   "id": "189d775b-60fe-4a43-bd06-3e159fd3f0d8",
   "metadata": {},
   "source": [
    "# ETL with Spark on EMR"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "id": "22df453c",
   "metadata": {},
   "outputs": [],
   "source": [
    "from pyspark.sql import SparkSession\n",
    "from pyspark.sql.types import StructType, StructField, DoubleType, StringType, IntegerType, DateType, TimestampType\n",
    "\n",
    "import pyspark.sql.functions as F"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "id": "c8ddf25d",
   "metadata": {},
   "outputs": [],
   "source": [
    "bucket = \"s3://zkan-dataset\"\n",
    "landing_zone = f\"{bucket}/landing\"\n",
    "cleaned_zone_csv = f\"{bucket}/cleaned\"\n",
    "cleaned_zone_parquet = f\"{bucket}/cleaned_parquet\""
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "id": "967f8dc3",
   "metadata": {},
   "outputs": [],
   "source": [
    "spark = SparkSession.builder \\\n",
    "    .appName(\"ETL\") \\\n",
    "    .getOrCreate()"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "id": "d63bfa2f",
   "metadata": {},
   "outputs": [],
   "source": [
    "data = spark.read.option(\"multiline\", \"true\").json(landing_zone)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "id": "f0a94871",
   "metadata": {},
   "outputs": [],
   "source": [
    "data.printSchema()"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "id": "0a80aace",
   "metadata": {},
   "outputs": [],
   "source": [
    "data.createOrReplaceTempView(\"staging_events\")"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "id": "7347e0c8",
   "metadata": {},
   "outputs": [],
   "source": [
    "table = spark.sql(\"\"\"\n",
    "    select\n",
    "        id\n",
    "        , type\n",
    "        , created_at\n",
    "        , to_date(created_at) as date\n",
    "        , year(created_at) as year\n",
    "        \n",
    "    from\n",
    "        staging_events\n",
    "\"\"\")"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "id": "917cf32e",
   "metadata": {},
   "outputs": [],
   "source": [
    "table.show()"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "id": "74af3629",
   "metadata": {},
   "outputs": [],
   "source": [
    "table.write.partitionBy(\"year\").mode(\"overwrite\").csv(cleaned_zone)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "id": "5fcbfd76",
   "metadata": {},
   "outputs": [],
   "source": [
    "table.write.partitionBy(\"year\").mode(\"overwrite\").parquet(cleaned_zone_parquet)"
   ]
  }
 ],
 "metadata": {
  "kernelspec": {
   "display_name": "Python 3 (ipykernel)",
   "language": "python",
   "name": "python3"
  },
  "language_info": {
   "codemirror_mode": {
    "name": "ipython",
    "version": 3
   },
   "file_extension": ".py",
   "mimetype": "text/x-python",
   "name": "python",
   "nbconvert_exporter": "python",
   "pygments_lexer": "ipython3",
   "version": "3.10.6"
  }
 },
 "nbformat": 4,
 "nbformat_minor": 5
}